In [4]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt 
import oct2py
import math
from mpl_toolkits.mplot3d import Axes3D


oc = oct2py.Oct2Py()

In [6]:
# Initial states (Bell Diagonal states)
b1 = bell_state('00')
b2 = bell_state('01')
b3 = bell_state('10')
b4 = bell_state('11')

#States of bath qubits
e1 = tensor(maximally_mixed_dm(2), maximally_mixed_dm(2))
e2 = tensor(maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2))

# Total initial state:
i1 = tensor(ket2dm(b1), e1, e2)

# Definitions
sx = sigmax()
sy = sigmay()
sz = sigmaz()

#Parameters for Hamiltonian (Couplngs etc)

#Coupling between two qubits
J_12 = 10

#Coupling between qubit and first layer of environment
J_23 = 20

#Coupling between first and second layer of environment 
J_34 = 30

#Chemical potential terms:
o_1 = 0
o_2 = 0
o_3 = 0
o_4 = 0


# Scrambling Time
#tau = 

# # First environement evolution time
# t = 0.1

In [7]:
# Function to calculate otoc
# Input must be a list or array of matrices corresponding to the states 
def otoc_calc(a):
    otoc=[]
    for i in range(0, len(a)):
        num = (a[i]*otoc_states[0]).tr()
        den = ((a[i]*a[i]).tr()*(otoc_states[0]*otoc_states[0]).tr())**(0.5)
        otoc_calc = num/den
        otoc.append(otoc_calc)   
    return otoc;

# Function to calculate otoc
# Input must be a list or array of matrices corresponding to the states 
def discord_calc(q):
    discord = []
    for i in range(0, len(discord_states)):
        disc_calc = oc.discord(discord_states[i], 200, 200)
        discord.append(disc_calc)
    return discord;

In [18]:
# Hamiltonians

# System
H_12 = J_12/4 *tensor(sz, sz, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2))

# Coupling between system and first environment
H_23 = J_23/4 *(tensor(qeye(2), sz, sz, qeye(2), qeye(2), qeye(2), qeye(2)) + tensor(qeye(2), sz, qeye(2), sz, qeye(2), qeye(2), qeye(2)))

# Coupling between two layers of environment
H_34 = J_34/4 *(tensor(qeye(2), qeye(2), sz, qeye(2), sz, qeye(2), qeye(2)) +
                 tensor(qeye(2), qeye(2), sz, qeye(2), qeye(2), sz, qeye(2)) +
                 tensor(qeye(2), qeye(2), sz, qeye(2), qeye(2), qeye(2), sz) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, sz, qeye(2), qeye(2)) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, qeye(2), sz, qeye(2)) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, qeye(2), qeye(2), sz))

In [19]:
# Initial psuedopure state (Wigner state)

zhi=8

sys_init = (0.25*(1-(zhi/8))*tensor(qeye(2), qeye(2))) + (zhi/8 * ket2dm(b3))



In [ ]:
discordlist = []
otoclist = []
discord_big = [sys_init.full()]
otoc_big = [sys_init]
for t in np.linspace(0.1, 1, 10):
    discord_states = [sys_init.full()]
    otoc_states = [sys_init]
    for tau in np.linspace(0.01, 1, 100):
        
        tau_list=[0]

        # Full initial state
        full_init = tensor(sys_init, maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2))

        # Time evolution

        # Evolution under coupling with first environment
        r1= mesolve(H_12+H_23, full_init, np.linspace(0, t, 500))
        # Saving final state to use as initial state in next step
        r2i = r1.states[len(r1.states)-1]

        # Scrambling (half)
        r2= mesolve(H_12+H_23+H_34, r2i, np.linspace(0, tau/2, 500))
        r3i = r2.states[len(r2.states)-1]

        # Pi pulse on system 
        r3 = mesolve(0.5*(tensor(sx, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2), qeye(2)) + tensor(qeye(2), sx, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2))), r3i, np.linspace(0, np.pi, 500))
        r4i = r3.states[len(r3.states)-1]

        # Scrambling (second half)
        r4 = mesolve(H_12+H_23+H_34, r4i, np.linspace(0, tau/2, 500))
        r5i = r4.states[len(r4.states)-1]


        # First environment coupling evolution
        r5 = mesolve(H_12+H_23, r5i, np.linspace(0, t, 500))
        r6i = r5.states[len(r5.states)-1]

        # Saving reduced state for calculating discord
        discord_states.append(r6i.ptrace([0,1]).full())

        # Saving reduced state for calculating OTOC
        otoc_states.append(r6i.ptrace([0,1]))
        tau_list.append(tau)
        
        print(t, tau)
    
    print(len(discord_states))
    print(len(otoc_states))
    discord_f = discord_calc(discord_states)
    otoc_f = otoc_calc(otoc_states)
    otoc_big.append(otoc_f)
    discord_big.append(discord_f)
    
    print(len(otoc_big), len(discord_big))
    
    
#     discord_states_big.append(discord_states[1::])
#     otoc_states_big.append(otoc_states[1::])

#     discordlist.append(discord_f)
#     otoclist.append(otoc_f)
    
#     print(len(discordlist))
#     print(len(otoclist))


    



    
        


In [ ]:
# Block of code for plotting discord from data saved from previous loop
tmr = np.linspace(0.1, 1, 15)


fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(1, len(discord_big)):
    ax.scatter(np.linspace(0.01, 1, 100), [tmr[i]]*100, np.array(discord_big[i][1::]).real, zdir='z', linestyle='dashed')
    ax.set_zlabel("Discord")
    ax.set_xlabel(r"$\tau$")
    ax.set_ylabel("t")
plt.savefig("discord_3D_sz-sz")
# ax.plot(xs=tau_list, ys=0.1*len(tau_list), zs=discord_big[1], zdir='z', label='ys=0, zdir=z', linestyle='dashed')



In [ ]:
# Block of code for plotting otoc from data saved from previous loop
tmr = np.linspace(0.1, 1, 15)


fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(1, len(discord_big)):
    ax.scatter(np.linspace(0.01, 1, 100), [tmr[i]]*100, np.array(otoc_big[i][1::]).real, zdir='z', linestyle='dashed')
    ax.set_zlabel("OTOC")
    ax.set_xlabel(r"$\tau$")
    ax.set_ylabel("t")
    
plt.savefig("otoc_3D_sz-sz")